In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Citeseer"
pubmed_dataset = Planetoid(root='', name='Pubmed')
data = pubmed_dataset[0].to(device)
print(data)

Processing...
Done!


Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [6]:
model = GCN(data.x.shape[1], pubmed_dataset.num_classes, [16]).to(device)

In [7]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 1.2401303052902222, Val Loss - 0.9865222573280334, Val Accuracy - 0.568
Epoch 20, Train Loss - 0.18857699632644653, Val Loss - 1.118511438369751, Val Accuracy - 0.75
Epoch 40, Train Loss - 0.03934972360730171, Val Loss - 1.7093532085418701, Val Accuracy - 0.768
Epoch 60, Train Loss - 0.021115321666002274, Val Loss - 1.634511113166809, Val Accuracy - 0.766
Epoch 80, Train Loss - 0.03220207244157791, Val Loss - 2.0352060794830322, Val Accuracy - 0.762
Epoch 100, Train Loss - 0.023112816736102104, Val Loss - 1.9987075328826904, Val Accuracy - 0.765
Epoch 120, Train Loss - 0.023848453536629677, Val Loss - 2.2929670810699463, Val Accuracy - 0.765
Epoch 140, Train Loss - 0.03926992043852806, Val Loss - 2.2801482677459717, Val Accuracy - 0.758
Epoch 160, Train Loss - 0.011819147504866123, Val Loss - 2.2437214851379395, Val Accuracy - 0.761
Epoch 180, Train Loss - 0.016679096966981888, Val Loss - 2.216891050338745, Val Accuracy - 0.769
Epoch 200, Train Loss - 0.0330237820

In [8]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.755
Initial Loss: 2.2523627281188965


In [ ]:
from collections import defaultdict 
from torch_geometric.utils import to_networkx

# Step 1: Initialize and run Metattack
attacker = Metattack(data, device=device)
attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask,
                         lambda_=0.)
attacker.reset()

#200% perturbation (Large Pool)
attacker.attack(num_budgets=2.0)

# Collect added edges from the single run
edges_to_add = list(attacker._added_edges.keys())


print(f"Number of added edges: {len(edges_to_add)}")
print(f"Sample: {edges_to_add[:5]}")


Peturbing graph...:   0%|          | 0/88648 [00:00<?, ?it/s]

In [9]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [10]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.15
budget = int(ptb_rate * initial_edge_count)

In [11]:
print(len(edges_to_add))
print(budget)

1296
341


In [12]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [13]:
splits = [0, 0.5, 0.7, 0.9]
split_dic = defaultdict(list)

In [14]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.0750
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/341 [00:00<?, ?it/s]

Final Accuracy: 0.5370
Accuracy Drop: 0.0220
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0375
Phase 1: Added 170 edges out of requested 170.


Peturbing graph...:   0%|          | 0/177 [00:00<?, ?it/s]

Final Accuracy: 0.5330
Accuracy Drop: 0.0260
0.7

--- Running split: 0.7 ---
Second phase perturbation rate: 0.0225
Phase 1: Added 238 edges out of requested 238.


Peturbing graph...:   0%|          | 0/107 [00:00<?, ?it/s]

Final Accuracy: 0.5360
Accuracy Drop: 0.0230
0.9

--- Running split: 0.9 ---
Second phase perturbation rate: 0.0075
Phase 1: Added 306 edges out of requested 306.


Peturbing graph...:   0%|          | 0/36 [00:00<?, ?it/s]

Final Accuracy: 0.5460
Accuracy Drop: 0.0130


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
